# Connect to Milvus

In [1]:
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema
)

host = '161.156.196.183'
port = '8080'
password = '4XYg2XK6sMU4UuBEjHq4EhYE8mSFO3Qq'
user = 'root'
server_pem_path =  './data/cert.pem'

connections.connect(alias = 'default',
                host = host,
                port = port,
                user = user,
                password = password,
                server_pem_path=server_pem_path,
                server_name = 'localhost',
                secure = True)

In [57]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Primary key
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=300,),
    FieldSchema(name="article", dtype=DataType.VARCHAR, max_length=2500,),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024,),
    FieldSchema(name="school_year", dtype=DataType.INT16, max_length=300,),
    FieldSchema(name="in_effect", dtype=DataType.VARCHAR, max_length=100,),
    FieldSchema(name="file_links", dtype=DataType.VARCHAR, max_length=300),
    FieldSchema(name="created_at", dtype=DataType.VARCHAR, max_length=200,),
    FieldSchema(name="updated_at", dtype=DataType.VARCHAR, max_length=200,),
]

schema = CollectionSchema(fields, "news schema")

In [58]:
collection_names = ['recruitment', 'timetable', 'scholarship', 'academic_affairs', 'events']

In [59]:
for col in collection_names:
    collection = Collection(col, schema)
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name='embedding', index_params=index_params)

In [60]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Primary key
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=300,),
    FieldSchema(name="article", dtype=DataType.VARCHAR, max_length=5000,),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024,),
    FieldSchema(name="school_year", dtype=DataType.INT16,),
    FieldSchema(name="in_effect", dtype=DataType.VARCHAR, max_length=100,),
    FieldSchema(name="created_at", dtype=DataType.VARCHAR, max_length=200,),
    FieldSchema(name="updated_at", dtype=DataType.VARCHAR, max_length=200,),
] #Missing page number

schema = CollectionSchema(fields, "news schema")

collection = Collection('student_handbook', schema)

index_params = {
    'metric_type':'L2',
    'index_type':"IVF_FLAT",
    'params':{"nlist":2048}
}
collection.create_index(field_name='embedding', index_params=index_params)

Status(code=0, message=)

In [61]:
connections._fetch_handler('default').list_collections()

['timetable',
 'student_handbook',
 'recruitment',
 'scholarship',
 'academic_affairs',
 'events']

In [55]:
for col in connections._fetch_handler('default').list_collections():
    connections._fetch_handler('default').drop_collection(col)

# Process data

In [62]:
from glob import glob

paths = glob('./data/thesis_rag_data/*')

In [63]:
import json 
data = []
for path in paths:
    with open(path, 'r', encoding='utf-8') as rstream:
        data = json.load(rstream)

    for d in data:
        d.pop('_id', None)
        d['school_year'] = int(d['school_year'])
    collection = Collection(path.split('\\')[-1].split('.')[0])
    collection.insert(data)


In [11]:
path = paths[4]
path.split('\\')[-1].split('.')[0]

'student_handbook'

In [7]:
import json
with open(path, 'r', encoding='utf-8') as rstream:
    data = json.load(rstream)

for d in data:
    d.pop('_id', None)
    d['school_year'] = int(d['school_year'])

In [ ]:
for col in collection_names:
    Collection(col).insert(data)

In [134]:
collection = Collection('timetable')

In [135]:
collection.insert(data)

(insert count: 8, delete count: 0, upsert count: 0, timestamp: 452156547920035842, success count: 8, err count: 0

***

In [80]:
# for c in connections._fetch_handler('default').list_collections():
#     Collection(c).drop()

***

In [65]:
import os
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from ibm_watsonx_ai.foundation_models import Embeddings
from dotenv import load_dotenv

In [66]:
load_dotenv('./data/.env')
my_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.environ['WATSONX_APIKEY'],
}

# model_id = 'sentence-transformers/all-minilm-l12-v2'
model_id = 'intfloat/multilingual-e5-large'
gen_parms = None
project_id = os.environ['WATSONX_PROJECT_ID']
space_id = None
verify = False

# Set the truncate_input_tokens to a value that is equal to or 
# less than the maximum allowed tokens for the embedding model that you are using. 
# If you don't specify this value and the input has more tokens than the model can process, 
# an error is generated.

embed_params = {
    EmbedParams.TRUNCATE_INPUT_TOKENS: 512,
}

model = Embeddings(
    model_id=model_id,
    credentials=my_credentials,
    params=embed_params,
    project_id=project_id,
    verify=verify
)

In [67]:
query="Khoa công nghệ thông tin là gì"

In [68]:
query_embeddings = model.embed_query(query)

col_name = "student_handbook"
collection = Collection(col_name)
collection.load()
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 5}
}
top_k = 3
output_fields = ['title', 'article'] #Section field to be added
results = collection.search(
    data=[query_embeddings],
    anns_field="embedding",
    param=search_params,
    limit=top_k,
    expr=None,
    output_fields=output_fields
)

In [70]:
results[0][0].to_dict()

{'id': 452583217926675827,
 'distance': 0.2861247956752777,
 'entity': {'title': 'Khoa công nghệ thông tin',
  'article': 'CHƯƠNG TRÌNH ĐÀO TẠO CHẤT LƯỢNG CAO \r\nNGÀNH CÔNG NGHỆ THÔNG TIN\r\nCÁC CHƯƠNG TRÌNH ĐÀO TẠO THEO ĐỀ ÁN – KHOA CÔNG NGHỆ THÔNG TIN\r\n Với mục tiêu nâng cao chất lượng đào tạo trình độ đại học nhằm đáp ứng \r\nnguồn nhân lực có tính cạnh tranh cao trên thị trường lao động trong thời kỳ hội nhập \r\nkinh tế khu vực và thế giới, Khoa Công nghệ thông tin triển khai tuyển sinh chương trình \r\nđào tạo Chất lượng cao ngành Công nghệ Thông tin đầu tiên vào năm 2013.\r\n Chương trình đào tạo được xây dựng phát triển trên cơ sở chương \r\ntrình đào tạo chính quy chuẩn, có tham khảo các khung chương trình đào \r\ntạo của các tổ chức nghề nghiệp lớn trên thế giới với yêu cầu về nâng cao \r\ntrình độ kiến thức nghề nghiệp, nội dung giảng dạy sâu rộng hơn, đề cao tư \r\nduy sáng tạo, tự học, tăng cường kỹ năng thực hành, kiến thức thực tế, phát \r\ntriển kỹ năng cá nhân và kh